In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import sympy as sy
from sympy.stats import Normal, cdf
from scipy.stats import multivariate_normal as mvn
from scipy.optimize import minimize


In [2]:
def binomialPut(St, K,r, sigma, T,n):
    dt=T/n
    u= np.exp(sigma*np.sqrt(dt))
    d=1/u
    p= (np.exp(r*dt)-d)/(u-d)
    
    #Final stock price and option
    Stock=(St*np.ones(n+1)*u**n)*(np.power(d*np.ones(n+1),np.arange(0,2*(n+1),2)))
    Optionvalue = np.maximum(K*np.ones(n+1) - Stock,0)
    
    for i in range(n,0,-1):
        Optionvalue = np.exp(-r*dt)*(p*Optionvalue[0:i]+(1-p)*Optionvalue[1:])
    return Optionvalue[0]

def americanPut(St, K,r, sigma, T,n):
    dt=T/n
    u= np.exp(sigma*np.sqrt(dt))
    d=1/u
    p= (np.exp(r*dt)-d)/(u-d)
    
    #Final stock price and option
    Stock=(St*np.ones(n+1)*u**n)*(np.power(d*np.ones(n+1),np.arange(0,2*(n+1),2)))
    Optionvalue = np.maximum(K*np.ones(n+1) - Stock,0)
    
    for i in range(n,0,-1):
        F1 = np.exp(-r*dt)*(p*Optionvalue[0:i]+(1-p)*Optionvalue[1:])
        F2 = np.maximum(K*np.ones(i)-(St*np.ones(i)*u**(i-1))*(np.power(d*np.ones(i),np.arange(0,2*(i),2))),0)
        Optionvalue = np.maximum(F1,F2)
    return Optionvalue[0]

In [3]:
def binomialCall(St, K,r, sigma, T,n):
    dt=T/n
    u= np.exp(sigma*np.sqrt(dt))
    d=1/u
    p= (np.exp(r*dt)-d)/(u-d)
    
    #Final stock price and option
    Stock=(St*np.ones(n+1)*u**n)*(np.power(d*np.ones(n+1),np.arange(0,2*(n+1),2)))
    Optionvalue = np.maximum(Stock -K*np.ones(n+1),0)
    
    for i in range(n,0,-1):
        Optionvalue = np.exp(-r*dt)*(p*Optionvalue[0:i]+(1-p)*Optionvalue[1:])
    return Optionvalue[0]

def americanCall(St, K,r, sigma, T,n):
    dt=T/n
    u= np.exp(sigma*np.sqrt(dt))
    d=1/u
    p= (np.exp(r*dt)-d)/(u-d)
    
    #Final stock price and option
    Stock=(St*np.ones(n+1)*u**n)*(np.power(d*np.ones(n+1),np.arange(0,2*(n+1),2)))
    Optionvalue = np.maximum(Stock -K*np.ones(n+1),0)
    
    for i in range(n,0,-1):
        F1 = np.exp(-r*dt)*(p*Optionvalue[0:i]+(1-p)*Optionvalue[1:])
        F2 = np.maximum((St*np.ones(i)*u**(i-1))*(np.power(d*np.ones(i),np.arange(0,2*(i),2))) -K*np.ones(i),0)
        Optionvalue = np.maximum(F1,F2)
    return Optionvalue[0]

In [4]:
d1 = lambda St, K,r, sigma, T,t: (np.log(St/K)+(r+0.5*sigma**2)*(T-t))/(sigma*np.sqrt(T-t))
d2 = lambda St, K,r, sigma, T,t: (np.log(St/K)+(r-0.5*sigma**2)*(T-t))/(sigma*np.sqrt(T-t))
PutBS = lambda St, K,r, sigma, T,t:K*np.exp(-r*(T-t))*st.norm.cdf(-d2(St,K,r,sigma,T,t)) -St*st.norm.cdf(-d1(St,K,r,sigma,T,t))
CallBS = lambda St, K,r, sigma, T,t:St*st.norm.cdf(d1(St,K,r,sigma,T,t)) - K*np.exp(-r*(T-t))*st.norm.cdf(d2(St,K,r,sigma,T,t))

In [5]:
def SOptCall(S,X1,X2,T1,T2,r,sigma):
    tau = T2-T1
    bounds = [[0,None]]
    apply_constraint1 = lambda S: CallBS(S,X2,r,sigma,tau,0)-X1
    my_constraints = ({'type':'eq', 'fun':apply_constraint1})
    a = minimize(CallBS,S,bounds = bounds, constraints = my_constraints, method = 'SLSQP', args = (X2,r,sigma,tau,0))
    return a.x[0]

def SOptPut(S,X1,X2,T1,T2,r,sigma):
    tau = T2-T1
    bounds = [[0,None]]
    apply_constraint1 = lambda S: PutBS(S,X2,r,sigma,tau,0)-X1
    my_constraints = ({'type':'eq', 'fun':apply_constraint1})
    a = minimize(PutBS,S,bounds = bounds, constraints = my_constraints, method = 'SLSQP', args = (X2,r,sigma,tau,0))
    return a.x[0]

In [6]:
St=500
K=700
T=2
sigma=.30
r=.10
n=20
PutBS(St,K,r,sigma,T,0),CallBS(St,K,r,sigma,T,0)

(131.24079333624655, 58.12926618165923)

In [7]:
americanPut(St, K,r, sigma, T,n),americanCall(St, K,r, sigma, T,n)

(200.0, 57.85326542408653)

In [8]:
def Call_call(S,X1,X2,T1,T2,r,sigma):
    SOpt = SOptCall(S,X1,X2,T1,T2,r,sigma)
    D1 = (np.log(S/SOpt)+(r+0.5*sigma**2)*(T1))/(sigma*np.sqrt(T1))
    D2 = (np.log(S/SOpt)+(r-0.5*sigma**2)*(T1))/(sigma*np.sqrt(T1))
    E1 = (np.log(S/X2)+(r+0.5*sigma**2)*(T2))/(sigma*np.sqrt(T2))
    E2 = (np.log(S/X2)+(r-0.5*sigma**2)*(T2))/(sigma*np.sqrt(T2))
    Corr = np.sqrt(T1/T2)
    dist = mvn(mean = np.array([0,0]), cov = np.array([[1,Corr],[Corr,1]]))
    N2_D1E1 = dist.cdf(np.array([D1,E1]))
    N2_D2E2 = dist.cdf(np.array([D2,E2]))
    ND2 = st.norm.cdf(D2, 0,1)
    return S*N2_D1E1-X2*np.exp(-r*T2)*N2_D2E2 -X1*np.exp(-r*T1)*ND2

def Call_put(S,X1,X2,T1,T2,r,sigma):
    SOpt = SOptPut(S,X1,X2,T1,T2,r,sigma)
    D1 = (np.log(S/SOpt)+(r+0.5*sigma**2)*(T1))/(sigma*np.sqrt(T1))
    D2 = (np.log(S/SOpt)+(r-0.5*sigma**2)*(T1))/(sigma*np.sqrt(T1))
    E1 = (np.log(S/X2)+(r+0.5*sigma**2)*(T2))/(sigma*np.sqrt(T2))
    E2 = (np.log(S/X2)+(r-0.5*sigma**2)*(T2))/(sigma*np.sqrt(T2))
    Corr = np.sqrt(T1/T2)
    dist = mvn(mean = np.array([0,0]), cov = np.array([[1,Corr],[Corr,1]]))
    N2_D1E1 = dist.cdf(np.array([-D1,-E1]))
    N2_D2E2 = dist.cdf(np.array([-D2,-E2]))
    ND2 = st.norm.cdf(-D2, 0,1)
    return -S*N2_D1E1+X2*np.exp(-r*T2)*N2_D2E2 -X1*np.exp(-r*T1)*ND2

def Put_call(S,X1,X2,T1,T2,r,sigma):
    SOpt = SOptCall(S,X1,X2,T1,T2,r,sigma)
    D1 = (np.log(S/SOpt)+(r+0.5*sigma**2)*(T1))/(sigma*np.sqrt(T1))
    D2 = (np.log(S/SOpt)+(r-0.5*sigma**2)*(T1))/(sigma*np.sqrt(T1))
    E1 = (np.log(S/X2)+(r+0.5*sigma**2)*(T2))/(sigma*np.sqrt(T2))
    E2 = (np.log(S/X2)+(r-0.5*sigma**2)*(T2))/(sigma*np.sqrt(T2))
    Corr = np.sqrt(T1/T2)
    dist = mvn(mean = np.array([0,0]), cov = np.array([[1,Corr],[Corr,1]]))
    N2_D1E1 = dist.cdf(np.array([-D1,E1]))
    N2_D2E2 = dist.cdf(np.array([-D2,E2]))
    ND2 = st.norm.cdf(-D2, 0,1)
    return -S*N2_D1E1+X2*np.exp(-r*T2)*N2_D2E2 +X1*np.exp(-r*T1)*ND2

def Put_put(S,X1,X2,T1,T2,r,sigma):
    SOpt = SOptPut(S,X1,X2,T1,T2,r,sigma)
    D1 = (np.log(S/SOpt)+(r+0.5*sigma**2)*(T1))/(sigma*np.sqrt(T1))
    D2 = (np.log(S/SOpt)+(r-0.5*sigma**2)*(T1))/(sigma*np.sqrt(T1))
    E1 = (np.log(S/X2)+(r+0.5*sigma**2)*(T2))/(sigma*np.sqrt(T2))
    E2 = (np.log(S/X2)+(r-0.5*sigma**2)*(T2))/(sigma*np.sqrt(T2))
    Corr = np.sqrt(T1/T2)
    dist = mvn(mean = np.array([0,0]), cov = np.array([[1,Corr],[Corr,1]]))
    N2_D1E1 = dist.cdf(np.array([D1,-E1]))
    N2_D2E2 = dist.cdf(np.array([D2,-E2]))
    ND2 = st.norm.cdf(D2, 0,1)
    return S*N2_D1E1-X2*np.exp(-r*T2)*N2_D2E2 +X1*np.exp(-r*T1)*ND2

In [12]:
T1,T2 = 0.5,2
r = 0.1
sigma = 0.3
X1 = 100
X2 = 200
tau = T2-T1
S = 200

In [15]:
Call_call(S,X1,X2,T1,T2,r,sigma),Call_put(S,X1,X2,T1,T2,r,sigma)

(64.92224573148437, 34.58910063167396)

In [11]:
Put_call(S,X1,X2,T1,T2,r,sigma),Put_put(S,X1,X2,T1,T2,r,sigma)

(23.953064175568652, 7.822658890325748)